# Practico Final

![Diagrama](Diagrama.png)

## Variables globales
Voy a hacer que algunas variables sean globales. Si bien en el flowchart estan encapsuladas en las funciones, tendria que usar objetos para que las variables queden mas ordenadas y me meteria en mucho lio

In [ ]:
temperatura_seteada = 60 #Temperatura a la que queremos que se caliente el agua
global T_media
global VT
global VT_nuevo#Vector temperatura para ir almacenando nuevos valores
T_media = 0#Se va a setear con la funcion Calculo T_media
load tabla_pendientes
global pendientes_t = tabla_pendientes

## ON-OFF
Esta funcion determina el estado de la pava resistencia de la pava
<br>`Temperatura seteada` es una constante que voy a insertar manualmente en un principio, vendria dada por el control de la pava
<br>`DT` Viene dado por la funcion `Calculo DT` y `T media` viene dada por la funcion `Calculo VT`
<br> El nucleo de la funcion es una comparacion entre `T media` que se va a llamar simplemente `T` y la temperatura seteada usando el $\Delta T$ para determinar si cortar o no

In [ ]:
function estado = on_off(temperatura_seteada)
    DT = calculo_DT();
    T = calculo_t_media();
    
    if T+DT >= temperatura_seteada
        estado=0;
    else
        estado=1;
    end
end

## Calculo $\Delta T$

Esta funcion va a tomar el valor ya seteado de la pendiente y compararlo con la tabla pre-seteada de relaciones entre pendiente y nivel de agua para sacar el $\Delta T$. En verdad el nivel de agua no lo necesito.

In [ ]:
function DT = calculo_DT()
    global pendientes_t
    pendiente = calculo_pendiente();
    x=1;
    while(pendiente < pendientes_t(x, 1))
        x++;
        if x > length(pendientes_t(:, 1)+2)
            printf("Falla en calculo de pendiente\n")
            return
        end
    end
    DT = pendientes_t(x, 2);
end

## Calculo pendiente
Aca sacamos mediante regresion lineal modificada la pendiente estimada del vector de temperaturas

In [ ]:
function pendiente = calculo_pendiente()
    global VT;
    X = linspace(0, 1, 12);#Intervalo de tiempo de un segundo
    Y = VT;#Vector temperaturas medidas en un segundo fraccionadas en 12 mediciones

    #Calculo valores para pendiente de regresion lineal
    n = length(X);
    x = sum(X);
    y = sum(Y);
    xy = sum(X .* Y);
    xx = sum(X .^ 2);
    xx2 = (sum(X))^2;
    med_x = (mean(X));
    med_y = (mean(Y));

    pendiente = (n*xy - x*y) / (n*xx - xx2);
end

## Calculo T Media
Función que calcula la temperatura media basandose en la variable global del vector de temperaturas `VT`

In [ ]:
function t_media = calculo_t_media()
    global VT
    t_media = mean(VT);
end

## Calculo Vector temperatura
Esta función calcula el vector de 12 mediciones de temperatura correspondientes a un segundo de ejecución. Para calcularlo recibe un valor de temperatura instantanea y lo acumula en un vector auxiliar que se va llenando. Cuando el vector tiene dimensión 12, entonces reemplaza variable global `VT`. No lo devuelve. De esa forma, actualiza por su cuenta la variable del vector de temperaturas el cual siempre esta disponible. Como se ejecuta una vez por medicion, el vector auxiliar tambien lo tengo que almacenar como variable global

In [ ]:
function calculo_VT(t_instantanea)
    global VT_nuevo;
    VT_nuevo(end+1) = t_instantanea;
    
    if length(VT_nuevo) > 11
        global VT
        VT = VT_nuevo;
        VT_nuevo = [];
    end
end

In [ ]:
for x=[1:length(aux)]
    calculo_VT(aux(x))
end

In [ ]:
aux = [31.6, 31.6, 31.7, 31.6, 31.8, 31.9, 31.7, 31.8, 32.0, 31.9, 31.9, 32.0, 32.2, 32.1, 32.3];

In [ ]:
on_off(48)

# Simulacion
Voy a simular el comportamiento de la pava en lo que seria la sonda de tempratura ya que no puedo correr el programa en la pava realmente

Vamos a probar de usar ecuación de Newton para sacar la temperatura del agua:

$$\frac{\partial T}{\partial t} = -k (T - T_m)$$

In [13]:
function resultado = heun(f, x_0, y_0, X, cant_pasos=1)
    h = abs((x_0-X) / cant_pasos);
    x = x_0;
    y = y_0;
    
    while x < X
        x1 = x+h;
        y1p = y + f(x, y)*h;
        y1 = y + h*(f(x, y) + f(x1, y1p)) / 2;
        x=x1;
        y=y1;
    end
    resultado = y;
end    

In [ ]:
T_m = 18400

In [7]:
f = @(T) -0.000023*(T-18400)

f =

@(T) -0.000023 * (T - 18400)



In [8]:
f(3)

ans = 0.4231


In [15]:
heun(f, 83, 46, 89, 1)

ans = 48.527
